In [43]:
pip install mediapipe opencv-python matplotlib numpy pandas


Note: you may need to restart the kernel to use updated packages.


In [44]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt


In [45]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils


In [46]:
class ExerciseCounter:
    def __init__(self):
        self.count = 0
        self.up = False

    def count_reps(self, angle, min_angle, max_angle):
        if angle > max_angle and not self.up:
            self.up = True
        elif angle < min_angle and self.up:
            self.count += 1
            self.up = False

def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint
    c = np.array(c)  # End point
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360-angle
    return angle


In [47]:
squat_counter = ExerciseCounter()

def detect_squats(frame, landmarks):
    # Knee, Hip, and Ankle landmarks
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

    angle = calculate_angle(left_hip, left_knee, left_ankle)
    squat_counter.count_reps(angle, min_angle=60, max_angle=160)
    return angle, squat_counter.count


In [48]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Squat Count Example
        angle, count = detect_squats(image, landmarks)
        cv2.putText(image, f'Squats: {count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow('Exercise Counter', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [49]:
motion_data = []

def track_motion(landmarks, frame_count):
    # Extract positions for specific joints (e.g., knees, ankles)
    left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
    motion_data.append((frame_count, left_knee.x, left_knee.y))

# Analyze motion data
motion_array = np.array(motion_data)
plt.plot(motion_array[:, 0], motion_array[:, 1], label="Left Knee X")
plt.plot(motion_array[:, 0], motion_array[:, 2], label="Left Knee Y")
plt.legend()
plt.show()


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
import pandas as pd

df = pd.DataFrame(motion_data, columns=["Frame", "Knee_X", "Knee_Y"])
df.to_csv("motion_analysis.csv", index=False)
